#Teste ANOVA com 2 níveis

##Questão de pesquisa

> A depressão maior está associada à quantidade de cigarros entre os fumantes adultos jovens atuais?

###População 
Jovens adultos fumantes

###Variáveis

*   **Variável Explanatória** (independente): \[*Deprimido/não deprimido*\]
categórica, com 2 níveis
*   **Variável de Resposta** (dependente)
Quantidade de fumo, cigarros fumados por mês (1- 2940)


###Hipóteses
*   **Hipótese Nula:** Não há relação entre depressão e consumo de cigarros

*   **Hipótese Alternativa:** A depressão influencia o consumo de cigarros


###Iniciando nosso programa
Importando as bibliotecas

In [1]:
# -*- coding: utf-8 -*-

import numpy
import pandas
import statsmodels.formula.api as smf
import statsmodels.stats.multicomp as multi 

pandas.set_option('mode.chained_assignment', None)

Importando o dataset Nesarc

In [3]:
data = pandas.read_csv('./datasets/nesarc_pds.csv', low_memory=False)

Transformando os campos de texto em números

In [4]:
#setting variables you will be working with to numeric
data['S3AQ3C1'] = pandas.to_numeric(data['S3AQ3C1'], errors='coerce')
data['S3AQ3B1'] = pandas.to_numeric(data['S3AQ3B1'], errors='coerce')
data['CHECK321'] = pandas.to_numeric(data['CHECK321'], errors='coerce')

Pegando a população: filtrando jovens que fumaram nos últimos 12 meses (CHECK321=1)

In [5]:
#subset data to young adults age 18 to 25 who have smoked in the past 12 months
sub1=data[(data['AGE']>=18) & (data['AGE']<=25) & (data['CHECK321']==1)]


Tratando dados ausentes

In [6]:
#SETTING MISSING DATA
sub1['S3AQ3B1']=sub1['S3AQ3B1'].replace(9, numpy.nan)
sub1['S3AQ3C1']=sub1['S3AQ3C1'].replace(99, numpy.nan)

Melhorando a codificação: alterando a ordem com recode1

In [7]:
#recoding number of days smoked in the past month
recode1 = {1: 30, 2: 22, 3: 14, 4: 5, 5: 2.5, 6: 1}
sub1['USFREQMO']= sub1['S3AQ3B1'].map(recode1)

#converting new variable USFREQMMO to numeric
sub1['USFREQMO']= pandas.to_numeric(sub1['USFREQMO'], errors='coerce')

Criando uma variável secundária multiplicando os dias fumados/mês e o número de cigarros/por dia

Número de cigarros fumados no mês passado

In [8]:
# Creating a secondary variable multiplying the days smoked/month and the number of cig/per day
sub1['NUMCIGMO_EST']=sub1['USFREQMO'] * sub1['S3AQ3C1']

sub1['NUMCIGMO_EST']= pandas.to_numeric(sub1['NUMCIGMO_EST'], errors='coerce')

Vamos testar os grupos criados:

In [9]:
ct1 = sub1.groupby('NUMCIGMO_EST').size()
print (ct1)

NUMCIGMO_EST
1.0       29
2.0       14
2.5       11
3.0       12
4.0        2
          ..
1050.0     1
1200.0    29
1800.0     2
2400.0     1
2940.0     1
Length: 66, dtype: int64


Hora de aplicar o Teste F de ANOVA
Função ordinary least squares - OLS 
- Parâmetros 
  *  *formula* =\[variável dependente\] ~ variáve categórica. 
   * Perceba que colocamos dentro de C() para sinalizar que MAJORDEPLIFE é categórica.
  * dataset (sub1)


In [10]:
# using ols function for calculating the F-statistic and associated p value
model1 = smf.ols(formula='NUMCIGMO_EST ~ C(MAJORDEPLIFE)', data=sub1)
results1 = model1.fit()
print (results1.summary())

                            OLS Regression Results                            
Dep. Variable:           NUMCIGMO_EST   R-squared:                       0.002
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     3.550
Date:                Tue, 12 Jul 2022   Prob (F-statistic):             0.0597
Time:                        21:56:56   Log-Likelihood:                -11934.
No. Observations:                1697   AIC:                         2.387e+04
Df Residuals:                    1695   BIC:                         2.388e+04
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
Intercept              312.8380 

Vamos fazer um novo dataset com apenas as nossas variáveis de interesse 
* NUMCIGMO_EST
* MAJOORDEPLIFE

In [11]:
sub2 = sub1[['NUMCIGMO_EST', 'MAJORDEPLIFE']].dropna()

print ('means for numcigmo_est by major depression status')
m1= sub2.groupby('MAJORDEPLIFE').mean()
print (m1)

print ('standard deviations for numcigmo_est by major depression status')
sd1 = sub2.groupby('MAJORDEPLIFE').std()
print (sd1)

means for numcigmo_est by major depression status
              NUMCIGMO_EST
MAJORDEPLIFE              
0               312.837989
1               341.375000
standard deviations for numcigmo_est by major depression status
              NUMCIGMO_EST
MAJORDEPLIFE              
0               269.002344
1               288.495118


#Teste ANOVA com mais de 2 níveis

##Questão de pesquisa

> A etnia está associada à quantidade de cigarros entre os fumantes adultos jovens atuais?

###População 
Jovens adultos fumantes

###Variáveis

*   **Variável Explanatória** (independente): 
\[*Deprimido/não deprimido*\]

1.   Branco
2.   Negro
3.   Índio Americano, nativo do Alasca
4.   Asiático ou havaiano
5.   Latino


categórica, com 2 níveis
*   **Variável de Resposta** (dependente)
Quantidade de fumo, cigarros fumados por mês (1- 2940)


###Hipóteses
*   **Hipótese Nula:** Não há relação entre etnia e consumo de cigarros

*   **Hipótese Alternativa:** A etnia influencia o consumo de cigarros


In [12]:
sub3 = sub1[['NUMCIGMO_EST', 'ETHRACE2A']].dropna()

model2 = smf.ols(formula='NUMCIGMO_EST ~ C(ETHRACE2A)', data=sub3).fit()
print (model2.summary())


                            OLS Regression Results                            
Dep. Variable:           NUMCIGMO_EST   R-squared:                       0.055
Model:                            OLS   Adj. R-squared:                  0.052
Method:                 Least Squares   F-statistic:                     24.40
Date:                Tue, 12 Jul 2022   Prob (F-statistic):           1.18e-19
Time:                        21:57:17   Log-Likelihood:                -11888.
No. Observations:                1697   AIC:                         2.379e+04
Df Residuals:                    1692   BIC:                         2.381e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept           368.7865      8.22

Agora vamos analisar os grupos

In [13]:
print ('means for numcigmo_est by major depression status')
m2= sub3.groupby('ETHRACE2A').mean()
print (m2)

print ('standard deviations for numcigmo_est by major depression status')
sd2 = sub3.groupby('ETHRACE2A').std()
print (sd2)


means for numcigmo_est by major depression status
           NUMCIGMO_EST
ETHRACE2A              
1            368.786528
2            259.273810
3            310.988095
4            244.258621
5            219.758258
standard deviations for numcigmo_est by major depression status
           NUMCIGMO_EST
ETHRACE2A              
1            281.430730
2            278.677392
3            260.116964
4            195.076441
5            220.859365


Resumo das etnias
Teste Post Hoc

In [14]:
mc1 = multi.MultiComparison(sub3['NUMCIGMO_EST'], sub3['ETHRACE2A'])
res1 = mc1.tukeyhsd()
print(res1.summary())

   Multiple Comparison of Means - Tukey HSD, FWER=0.05   
group1 group2  meandiff p-adj    lower     upper   reject
---------------------------------------------------------
     1      2 -109.5127  0.001 -164.6441  -54.3814   True
     1      3  -57.7984 0.6251 -172.5914   56.9945  False
     1      4 -124.5279 0.0051 -222.9229  -26.1329   True
     1      5 -149.0283  0.001   -194.89 -103.1665   True
     2      3   51.7143 0.7555  -71.6021  175.0307  False
     2      4  -15.0152    0.9  -123.233   93.2026  False
     2      5  -39.5156 0.4492 -103.8025   24.7714  False
     3      4  -66.7295 0.7058 -214.5437   81.0848  False
     3      5  -91.2298 0.2269 -210.6902   28.2305  False
     4      5  -24.5004    0.9 -128.3027    79.302  False
---------------------------------------------------------
